# MNIST classifier using CNN

## Authors:

Nicola Rocco de Luna Pedulla - nrlp


Renan de Freitas Lins - rfl3

Ricardo Rossiter Barioni - rrb

Victor Miranda de Melo - vmm

## Firstly, import the required dependencies

GPU is not being used

In [86]:
from DeepLearningPython35.network3 import load_data_shared, Network, ConvPoolLayer, FullyConnectedLayer, SoftmaxLayer, ReLU, size
import theano.tensor as T
import theano
import numpy as np

In [25]:
training_data, validation_data, test_data = load_data_shared()
mini_batch_size = 10

In [94]:
def get_subset(data, l):
    
    x =  data[0].get_value(borrow=True)[:l]
    y = data[1].eval()[:l]

    shared_x = theano.shared( np.asarray(x, dtype=theano.config.floatX), borrow=True)
    shared_y = theano.shared( np.asarray(y, dtype=theano.config.floatX), borrow=True)
    return (shared_x, T.cast(shared_y, "int32"))

In [95]:

training_data_subset = get_subset(training_data, 10)
test_data_subset = get_subset(test_data, 10)
validation_data_subset = get_subset(validation_data, 10)


In [96]:
net = Network([
    ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28),
                  filter_shape=(20, 1, 5, 5),
                  poolsize=(2, 2),
                  activation_fn=ReLU),
    ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12),
                  filter_shape=(40, 20, 5, 5),
                  poolsize=(2, 2),
                  activation_fn=ReLU),
    FullyConnectedLayer(n_in=40*4*4, n_out=100, activation_fn=ReLU),
    SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data_subset, 1, mini_batch_size, 0.03, validation_data_subset, test_data_subset, lmbda=0.1)

Training mini-batch number 0
Epoch 0: validation accuracy 0.00%
This is the best validation accuracy to date.
The corresponding test accuracy is 20.00%
Finished training network.
Best validation accuracy of 0.00% obtained at iteration 0
Corresponding test accuracy of 20.00%
